# 008 Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies)

In [34]:
spark

org.apache.spark.sql.SparkSession@53c26b8f

In [35]:
import org.apache.spark.sql.functions._

In [36]:
import org.apache.spark.sql.types._

In [37]:
import org.apache.spark.sql.expressions.Window

In [38]:
val spark=SparkSession.builder().appName("movies app").master("local[*]").getOrCreate()

spark = org.apache.spark.sql.SparkSession@53c26b8f


org.apache.spark.sql.SparkSession@53c26b8f

In [39]:
import spark.implicits._

# UDFs

In [40]:
val getYear=udf((year:String)=>{
    val p="\\d{4}".r
    val yr=p.findFirstMatchIn(year).getOrElse("").toString
    if(yr!=""){
        yr.toInt
    }
    else{
        0
    }
})

getYear = UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))

# loading movies

In [41]:
val Movie=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Movie.csv")
.dropDuplicates()

Movie = [index: string, MID: string ... 4 more fields]


[index: string, MID: string ... 4 more fields]

# printing table structure to know what are the columns and it's data type

In [42]:
Movie.printSchema

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- num_votes: string (nullable = true)



# Displaying first two records to see sample data
# observation 1: year,rating,num_votes,index columns data types should be Integer
# observation 2: also we need do trim to remove any extra spaces

In [43]:
Movie.show(2)

+-----+---------+-----------------+----+------+---------+
|index|      MID|            title|year|rating|num_votes|
+-----+---------+-----------------+----+------+---------+
|   85|tt3224288|Beyond the Clouds|2017|   7.0|     1123|
|  913|tt2178508|   Son of Sardaar|2012|   4.0|     7895|
+-----+---------+-----------------+----+------+---------+
only showing top 2 rows



# displaying total records in the table

In [44]:
Movie.count

3475

# as per observation need to change the data type

In [45]:
val movies=Movie.withColumn("year",getYear(Movie("year")).cast(IntegerType))
.withColumn("title",trim(Movie("title")))
.withColumn("MID",trim(Movie("MID")))
.withColumn("rating",trim(Movie("rating")).cast(DoubleType))
.withColumn("num_votes",trim(Movie("num_votes")).cast(LongType))
.withColumn("index",trim(Movie("index")).cast(IntegerType))
.dropDuplicates("MID")

movies = [index: int, MID: string ... 4 more fields]


[index: int, MID: string ... 4 more fields]

# now print the schema to verify columns and it's data type

In [46]:
movies.printSchema

root
 |-- index: integer (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- rating: double (nullable = true)
 |-- num_votes: long (nullable = true)



# print the total number of records in the table after data cleaning

In [47]:
movies.count

3475

# show the first two records as a sample data

In [48]:
movies.show(2)

+-----+---------+--------------------+----+------+---------+
|index|      MID|               title|year|rating|num_votes|
+-----+---------+--------------------+----+------+---------+
| 3008|tt0036077|              Kismet|1943|   7.5|       71|
| 2969|tt0102701|Prahaar: The Fina...|1991|   7.9|     1696|
+-----+---------+--------------------+----+------+---------+
only showing top 2 rows



# Load the Person.csv file and it has header and store theses records in Person variable

In [50]:
val Person=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Person.csv")

+-----+---------+---------------+------+
|index|      PID|           Name|Gender|
+-----+---------+---------------+------+
|    0|nm0000288| Christian Bale|  Male|
|    1|nm0000949| Cate Blanchett|Female|
+-----+---------+---------------+------+
only showing top 2 rows

root
 |-- index: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



Person = [index: string, PID: string ... 2 more fields]
p1Count = 38285


38285

# Print the schema of Person variable,first two records and it's count

In [51]:
Person.show(2)
Person.printSchema
val p1Count=Person.count

+-----+---------+---------------+------+
|index|      PID|           Name|Gender|
+-----+---------+---------------+------+
|    0|nm0000288| Christian Bale|  Male|
|    1|nm0000949| Cate Blanchett|Female|
+-----+---------+---------------+------+
only showing top 2 rows

root
 |-- index: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p1Count = 38285


38285

# Data Cleaning
# change the data types as per data and also remove extra spaces from data

In [52]:
val persons=Person.withColumn("PID",trim(Person("PID")))
.withColumn("Name",trim(Person("Name")))
.withColumn("Gender",trim(Person("Gender")))
.withColumn("index",trim(Person("index")).cast(IntegerType))
.dropDuplicates("PID")

persons = [index: int, PID: string ... 2 more fields]


[index: int, PID: string ... 2 more fields]

# show the sample data, schema and it's count of variable persons

In [53]:
persons.show(2)
persons.printSchema
val p2Count=persons.count

+-----+---------+-------------+------+
|index|      PID|         Name|Gender|
+-----+---------+-------------+------+
|15793|nm0022343|       Alpana|Female|
| 3121|nm0028416|Martin Andris|  Male|
+-----+---------+-------------+------+
only showing top 2 rows

root
 |-- index: integer (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p2Count = 37566


37566

# print the difference between Person and persons records count

In [56]:
val diff=p1Count-p2Count
println(diff+" duplicates found")

719 duplicates found


diff = 719


719

# Load the M_Director.csv file and it's has header

In [57]:
val M_Director=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/M_Director.csv")

M_Director = [index: string, MID: string ... 2 more fields]


[index: string, MID: string ... 2 more fields]

# display sample data, schema and count M_Director variable

In [59]:
M_Director.printSchema
M_Director.show(2)
val m_d1Count=M_Director.count

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: string (nullable = true)

+-----+---------+---------+---+
|index|      MID|      PID| ID|
+-----+---------+---------+---+
|    0|tt2388771|nm0785227|  0|
|    1|tt5164214|nm0002657|  1|
+-----+---------+---------+---+
only showing top 2 rows



m_d1Count = 3475


3475

# Data cleaning for M_Director

# remove duplicate records, change data type from string to appropriate data type and perform trim operation on each column separately

In [61]:
val m_directors=M_Director.withColumn("ID",trim(col("ID")).cast(LongType))
.withColumn("index",trim(col("index")).cast(LongType))
.withColumn("MID",trim(col("MID")))
.withColumn("PID",trim(col("PID")))
.dropDuplicates("MID","PID")
.dropDuplicates("ID")
.dropDuplicates

m_directors = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

# display sample data, schema and count m_director variable

In [63]:
m_directors.printSchema
m_directors.show(2)
val m_d2Count=m_directors.count

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)

+-----+---------+---------+---+
|index|      MID|      PID| ID|
+-----+---------+---------+---+
|   26|tt5074352|nm4318159| 26|
|   29|tt1098327|nm0939128| 29|
+-----+---------+---------+---+
only showing top 2 rows



m_d2Count = 3475


3475

# Display Count difference between M_Director and m_director variables

In [64]:
val diff_m_directors_count=m_d1Count-m_d2Count
println(diff_m_directors_count+" d")

0 d


diff_m_directors_count = 0


0

# Loading and cleaning M_Cast table

In [65]:
val M_Cast=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/M_Cast.csv")

M_Cast = [index: string, MID: string ... 2 more fields]


[index: string, MID: string ... 2 more fields]

# Sample data from M_Cast table

In [66]:
M_Cast.printSchema
M_Cast.show(2)
val m_c1Count=M_Cast.count

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: string (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|    0|tt2388771| nm0000288|  0|
|    1|tt2388771| nm0000949|  1|
+-----+---------+----------+---+
only showing top 2 rows



m_c1Count = 82837


82837

# Cleaning data m_cast

In [67]:
val m_cast=M_Cast.withColumn("ID",trim(col("ID")).cast(LongType))
.withColumn("index",trim(col("index")).cast(LongType))
.withColumn("MID",trim(col("MID")))
.withColumn("PID",trim(col("PID")))
.dropDuplicates("MID","PID")
//.dropDuplicates("MID","PID","ID")
.dropDuplicates("ID")
.dropDuplicates()

m_cast = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

# Sample data after data cleaning for m_cast

In [68]:
m_cast.printSchema
m_cast.show(2)
val m_c2Count=m_cast.count

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|   26|tt2388771|nm10302077| 26|
|   29|tt2388771|nm10302080| 29|
+-----+---------+----------+---+
only showing top 2 rows



m_c2Count = 82837


82837

# Print difference of total count after and before data cleaning

In [69]:
val diff_m_cast_count=m_c1Count-m_c2Count
println(diff_m_cast_count+" d")

0 d


diff_m_cast_count = 0


0

# now modifying column name from PID to MPID of m_director (To avoid conflicts while joining another, if we have same column name)

In [70]:
val m_dirs=m_directors.withColumnRenamed("PID","DPID")

m_dirs = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

In [76]:
val actorsWithMovies=persons.join(m_cast,"PID").dropDuplicates().join(movies,"MID")
.select("MID","PID","year","Name","Gender")

actorsWithMovies = [MID: string, PID: string ... 3 more fields]


[MID: string, PID: string ... 3 more fields]

In [81]:
actorsWithMovies.orderBy("PID").show(10)

+---------+---------+----+-----------------+------+
|      MID|      PID|year|             Name|Gender|
+---------+---------+----+-----------------+------+
|tt0053126|nm0000002|1959|    Lauren Bacall|Female|
|tt0087892|nm0000027|1984|    Alec Guinness|  Male|
|tt0046427|nm0000039|1953|     Deborah Kerr|Female|
|tt0046427|nm0000042|1953|        Alan Ladd|  Male|
|tt0066070|nm0000047|1970|     Sophia Loren|Female|
|tt0031580|nm0000073|1939|   Shirley Temple|Female|
|tt0075860|nm0000076|1977|François Truffaut|  null|
|tt0327437|nm0000092|2004|      John Cleese|  Male|
|tt0120102|nm0000093|1997|        Brad Pitt|  null|
|tt4977530|nm0000096|2017| Gillian Anderson|Female|
+---------+---------+----+-----------------+------+
only showing top 10 rows



In [140]:
val unemployed_actors_step1=actorsWithMovies.sort("PID")
.withColumn("diff",(col("year")-(lag("year",1) over Window.partitionBy("PID").orderBy("PID","year")))>3)
.na.drop(col("diff"))

Name: Unknown Error
Message: <console>:65: error: overloaded method value drop with alternatives:
  (minNonNulls: Int)org.apache.spark.sql.DataFrame <and>
  (cols: Seq[String])org.apache.spark.sql.DataFrame <and>
  (cols: Array[String])org.apache.spark.sql.DataFrame <and>
  (how: String)org.apache.spark.sql.DataFrame
 cannot be applied to (org.apache.spark.sql.Column)
       .na.drop(col("diff"))
           ^

StackTrace: 

In [122]:
m_cast.printSchema

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)



# we need drop duplicates after applying filter condition to avoid getting empty results since duplicates records failed to satify the our filter condition

# following query will not gives resutls, since we are dropping duplicates before appying filter condition
unemployed_actors_step1.dropDuplicates("PID").filter($"diff"===true).sort("PID","year").show(100)

In [133]:
unemployed_actors_step1.filter($"diff"===true).dropDuplicates("PID").sort("PID","year").show(100)

+---------+---------+----+--------------------+------+----+
|      MID|      PID|year|                Name|Gender|diff|
+---------+---------+----+--------------------+------+----+
|tt3823392|nm0000193|2018|          Demi Moore|Female|true|
|tt1606378|nm0000246|2013|        Bruce Willis|  Male|true|
|tt0848228|nm0000375|2012|   Robert Downey Jr.|  Male|true|
|tt3498820|nm0000673|2016|        Marisa Tomei|Female|true|
|tt5816682|nm0001132|2017|          Judi Dench|Female|true|
|tt5164214|nm0001162|2018|       Griffin Dunne|  Male|true|
|tt0099467|nm0001408|1990|       Shekhar Kapur|  Male|true|
|tt0848228|nm0001765|2012|  Harry Dean Stanton|  Male|true|
|tt0086034|nm0001934|1983|          Kabir Bedi|  Male|true|
|tt0986213|nm0002043|2007|       Madhuri Dixit|Female|true|
|tt1647668|nm0002326|2014|         Al Sapienza|  Male|true|
|tt0371735|nm0003504|2003|         Alka Yagnik|  null|true|
|tt1974382|nm0003817|2015|Michael Clarke Du...|  Male|true|
|tt0073569|nm0003987|1975|  Mohandas K. 

# Validation 1

In [130]:
unemployed_actors_step1.filter($"PID"==="nm0000193").sort("PID","year").show(100)

+---------+---------+----+----------+------+----+
|      MID|      PID|year|      Name|Gender|diff|
+---------+---------+----+----------+------+----+
|tt0115641|nm0000193|1996|Demi Moore|Female|null|
|tt3823392|nm0000193|2018|Demi Moore|Female|true|
+---------+---------+----+----------+------+----+



In [131]:
actorsWithMovies.filter($"PID"==="nm0000193").sort("PID","year").show(100)

+---------+---------+----+----------+------+
|      MID|      PID|year|      Name|Gender|
+---------+---------+----+----------+------+
|tt0115641|nm0000193|1996|Demi Moore|Female|
|tt3823392|nm0000193|2018|Demi Moore|Female|
+---------+---------+----+----------+------+



# Validation 2

In [134]:
unemployed_actors_step1.filter($"PID"==="nm0004472").sort("PID","year").show(100)

+---------+---------+----+------------+------+----+
|      MID|      PID|year|        Name|Gender|diff|
+---------+---------+----+------------+------+----+
|tt0246204|nm0004472|1989|Nutan Prasad|  Male|null|
|tt0278274|nm0004472|2000|Nutan Prasad|  Male|true|
+---------+---------+----+------------+------+----+



In [135]:
actorsWithMovies.filter($"PID"==="nm0004472").sort("PID","year").show(100)

+---------+---------+----+------------+------+
|      MID|      PID|year|        Name|Gender|
+---------+---------+----+------------+------+
|tt0246204|nm0004472|1989|Nutan Prasad|  Male|
|tt0278274|nm0004472|2000|Nutan Prasad|  Male|
+---------+---------+----+------------+------+



# Validation 3

In [136]:
unemployed_actors_step1.filter($"PID"==="nm0015000").sort("PID","year").show(100)

+---------+---------+----+----+------+-----+
|      MID|      PID|year|Name|Gender| diff|
+---------+---------+----+----+------+-----+
|tt0215515|nm0015000|1952|Ajit|  Male| null|
|tt0050758|nm0015000|1957|Ajit|  Male| true|
|tt0156606|nm0015000|1965|Ajit|  Male| true|
|tt0061046|nm0015000|1966|Ajit|  Male|false|
|tt0158449|nm0015000|1969|Ajit|  Male|false|
|tt0064842|nm0015000|1969|Ajit|  Male|false|
|tt0156641|nm0015000|1970|Ajit|  Male|false|
|tt0148266|nm0015000|1970|Ajit|  Male|false|
|tt0237395|nm0015000|1971|Ajit|  Male|false|
|tt0066766|nm0015000|1971|Ajit|  Male|false|
|tt0367921|nm0015000|1973|Ajit|  Male|false|
|tt0069871|nm0015000|1973|Ajit|  Male|false|
|tt0070935|nm0015000|1973|Ajit|  Male|false|
|tt0070947|nm0015000|1973|Ajit|  Male|false|
|tt0069751|nm0015000|1973|Ajit|  Male|false|
|tt0070253|nm0015000|1973|Ajit|  Male|false|
|tt0155805|nm0015000|1974|Ajit|  Male|false|
|tt0216110|nm0015000|1975|Ajit|  Male|false|
|tt0158356|nm0015000|1975|Ajit|  Male|false|
|tt0268216

In [137]:
actorsWithMovies.filter($"PID"==="nm0015000").sort("PID","year").show(100)

+---------+---------+----+----+------+
|      MID|      PID|year|Name|Gender|
+---------+---------+----+----+------+
|tt0215515|nm0015000|1952|Ajit|  Male|
|tt0050758|nm0015000|1957|Ajit|  Male|
|tt0156606|nm0015000|1965|Ajit|  Male|
|tt0061046|nm0015000|1966|Ajit|  Male|
|tt0064842|nm0015000|1969|Ajit|  Male|
|tt0158449|nm0015000|1969|Ajit|  Male|
|tt0156641|nm0015000|1970|Ajit|  Male|
|tt0148266|nm0015000|1970|Ajit|  Male|
|tt0237395|nm0015000|1971|Ajit|  Male|
|tt0066766|nm0015000|1971|Ajit|  Male|
|tt0070935|nm0015000|1973|Ajit|  Male|
|tt0069751|nm0015000|1973|Ajit|  Male|
|tt0367921|nm0015000|1973|Ajit|  Male|
|tt0070253|nm0015000|1973|Ajit|  Male|
|tt0069871|nm0015000|1973|Ajit|  Male|
|tt0070947|nm0015000|1973|Ajit|  Male|
|tt0155805|nm0015000|1974|Ajit|  Male|
|tt0216110|nm0015000|1975|Ajit|  Male|
|tt0158356|nm0015000|1975|Ajit|  Male|
|tt0268216|nm0015000|1976|Ajit|  Male|
|tt0214832|nm0015000|1976|Ajit|  Male|
|tt0076167|nm0015000|1977|Ajit|  Male|
|tt0157337|nm0015000|1978